In [1]:
import jax
from jax import jit
from jax import lax
from jax import vmap
import jax.numpy as jnp

from functools import partial

jax.config.update('jax_enable_x64', True)

In [2]:
import math
import numpy as np
import plotly.express as px
import IPython
import matplotlib.pyplot as plt 
import ipywidgets as widgets

%config InlineBackend.figure_formats = ['svg']

In [3]:
from jax_control_algorithms.trajectory_optimization import Solver, constraint_geq, constraint_leq, unpack_res
from jax_control_algorithms.ui import manual_investigate
from jax_control_algorithms.common import rk4

# Flow model

In [4]:
def make_trajectories(n_steps, P_max_src_1 = 100, P_max_src_2 = 10, balance=0.0):
    P_src_1 = P_max_src_1 * jnp.concatenate(( jnp.zeros(10), jnp.sin( jnp.linspace(0, math.pi, n_steps-20 )), jnp.zeros(10)  ))
    P_src_2 = P_max_src_2 * jnp.concatenate(( jnp.zeros(10), jnp.sin( jnp.linspace(0, math.pi, n_steps-20 )), jnp.zeros(10)  ))
    
    b = jnp.linspace(-balance, balance, n_steps)
    price  = b + jnp.concatenate(( jnp.ones(10), 1-jnp.sin( jnp.linspace(0, math.pi, n_steps-20 )), jnp.ones(10)  ))
    
    return P_src_1, P_src_2, price
    

def problem_def_powerflow(n_steps = 50):
        
    def model(x, u, k, theta):
        
        bat_1, bat_2 = x[0], x[1]
        P_bat_1, P_bat_2 = u[0], u[1]
        
        c_1, c_2, P_src_1, P_src_2, reward = theta['c_1'], theta['c_2'], theta['P_src_1'], theta['P_src_2'], theta['reward']
                
        #
        P_transmission = P_src_1[k] + P_bat_1
        P_yield = P_transmission + P_src_2[k] + P_bat_2
        
        #
        bat_1_dot = 1/c_1 * ( -P_bat_1 )
        bat_2_dot = 1/c_2 * ( -P_bat_2 )
        
        x_dot = jnp.array([
            bat_1_dot, bat_2_dot
        ])
 
        #
        R = reward[k] * P_yield
        J = - 0.01*R
        
        
        deb1 = P_src_2[k]
        deb2 = P_bat_2
                 
        return x_dot, P_transmission, P_yield, J, R, deb1, deb2

    def f(x, u, k, theta):
        x_dot, P_transmission, P_yield, J, R, _, _ = model(x, u, k, theta)
        return x_dot

    def g(x, u, k, theta):
        x_dot, P_transmission, P_yield, J, R, deb1, deb2 = model(x, u, k, theta)
        return P_transmission, P_yield, R, deb1, deb2

    def running_cost(x, u, k, theta):
        
        x_dot, P_transmission, P_yield, J, R, _, _ = model(x, u, k, theta)
        return J
        
    def terminal_state_eq_constraints(x_final, theta):
        bat_1, bat_2 = x_final

        return jnp.array([
            0.2,
            0.1
        ])
    
    def inequ_constraints(x, u, k, theta):
        
        bat_1,   bat_2   = x[:,0], x[:,1]
        P_bat_1, P_bat_2 = u[:,0], u[:,1]
        
        # constraints
        c_ineq = jnp.array([
            constraint_geq( bat_1, 0 ),
            constraint_leq( bat_1, 1 ),
            
            constraint_geq( bat_2, 0 ),
            constraint_leq( bat_2, 1 ),
            
            constraint_geq( P_bat_1, -10 ),
            constraint_leq( P_bat_1,  10 ),
            
            constraint_geq( P_bat_2, -50 ),
            constraint_leq( P_bat_2,  50 ),
        ])
        
        # x_dot, P_transmission, P_yield, J, c_ineq = model(x, u, k, theta)
        return c_ineq
        
    def make_guess(x0, theta):
        U_guess = jnp.zeros( (n_steps, 2) )

        X_guess = jnp.vstack((
            jnp.linspace(x0[0], x0[0], n_steps ),
            jnp.linspace(x0[1], x0[1], n_steps ),
        )).T
        
        return X_guess, U_guess
    
    
    P_src_1, P_src_2, reward = make_trajectories(n_steps, 100, 10)
    
    theta = { 
        'c_1'     : 200, 
        'c_2'     : 2000, 
        'P_src_1' : P_src_1, 
        'P_src_2' : P_src_2,
        'reward'  : reward
    }
    
    c_1_init, c_2_init = 0.5, 0.5
    x0 = jnp.array([ c_1_init, c_2_init ])
    
    #
    f_dscr = rk4(f, dt=1.0)

    return f_dscr, g, running_cost, None, inequ_constraints, theta, x0, make_guess



f_dscr, g, running_cost, terminal_state_eq_constraints, inequ_constraints, theta, x0, make_guess = problem_def_powerflow(n_steps = 50)

In [5]:
def plot_flow(X_opt, U_opt, system_outputs, theta):

    # prepare data
    P_transmission, P_yield, R, deb1, deb2 = system_outputs # unpack output variable (return of function g)

    _, _, P_src_1, P_src_2, reward = theta['c_1'], theta['c_2'], theta['P_src_1'], theta['P_src_2'], theta['reward']

    P_bat_1, P_bat_2 = U_opt[:,0], U_opt[:,1]
    soc_1, soc_2     = X_opt[:,0], X_opt[:,1]

    # make time vectors
    time  = jnp.linspace(0, soc_1.shape[0]-1, soc_1.shape[0])
    time2 = jnp.linspace(0, P_bat_1.shape[0]-1, P_bat_1.shape[0])

    # Create a figure and two subplots
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, sharex=True, figsize=(12, 5))

    ax1.plot(P_transmission, label='P_transmission' )
    ax1.plot(P_yield, label='P_yield' )
    ax1.plot(P_src_1, '-', label='P_src_1' )
    ax1.plot(P_src_2, '-', label='P_src_2' )
    ax1.legend()
    ax1.set_ylabel('power')

    ax2.plot(time2, P_bat_1, '-+', label='power bat 1')
    ax2.plot(time2, P_bat_2, '-+',  label='power bat 2')
    ax2.set_ylabel('soc %')
    ax2.legend()

    ax3.plot(time, soc_1, '-+',  label='soc bat 1')
    ax3.plot(time, soc_2, '-+',  label='soc bat 2')
    ax3.set_ylabel('soc %')
    ax3.legend()

    ax4.plot(time2, 100*reward, '-+',  label='100 * price')
    ax4.plot(time2, R, '-+',  label='R')
    ax4.set_ylabel('')
    ax4.legend()

    # Add a title to the entire figure
    fig.suptitle('time series')

    # Show the plot
    plt.show()    

In [6]:
comp_secs_in_one_h = 5 * 1000 * 4 # secs per hour
secs_in_one_h = 60 * 60

comp_secs_in_one_h / secs_in_one_h


5.555555555555555

In [7]:
sliders = {
        'c_1'           : widgets.FloatSlider(min=100, max=400,  step=1.0, value=200,  description='c_1'),
        'c_2'           : widgets.FloatSlider(min=100, max=8000, step=1.0, value=2000, description='c_2'),
        'P_src_1'       : widgets.FloatSlider(min=10, max=100, step=1.0, value=100, description='P_src_1'),
        'P_src_2'       : widgets.FloatSlider(min=1, max=10, step=1.0,   value=10, description='P_src_2'),
        'price_balance' : widgets.FloatSlider(min=-1, max=1, step=0.01, value=0.0, description='price_balance'),
}

solver = Solver( partial(problem_def_powerflow, n_steps = 50), use_continuation=False )

def set_theta_fn(solver, c_1, c_2, P_src_1, P_src_2, price_balance):
        solver.theta['c_1'] = c_1
        solver.theta['c_2'] = c_2

        P_src_1_, P_src_2_, price = make_trajectories(solver.n_steps, P_src_1, P_src_2, price_balance)

        solver.theta['P_src_1'] = P_src_1_
        solver.theta['P_src_2'] = P_src_2_
        solver.theta['reward']  = price
        
#solver.tol_inner = 0.001
#solver.eq_tol = 0.001

In [8]:
ui, output_box, print_output, plot_output = manual_investigate( solver, sliders, set_theta_fn, plot_flow )
display(ui, output_box)

GridBox(children=(FloatSlider(value=200.0, description='c_1', max=400.0, min=100.0, step=1.0), FloatSlider(val…

# Unit test: Compute parameter sweep

In [9]:
price_balance_start = -0.1
price_balance_end   = 0.1

solver = Solver( partial(problem_def_powerflow, n_steps = 50), use_continuation=False )
#solver.verbose = False
solver.theta['price_balance'] = price_balance_start
solver.run() # compile


print('loop')
trace_parameter_sweep = []
price_balances = jnp.linspace(price_balance_start, price_balance_end, 3) # note: do not compute too much as otherwise CI will timeout

for price_balance in price_balances:
    
    solver.theta['price_balance'] = price_balance
    
    solver_return = solver.run()
    X_opt, U_opt, system_outputs, res = solver_return
    is_converged, c_eq, c_ineq, trace, n_iter = unpack_res(res)
    assert is_converged
    
    trace_parameter_sweep.append(solver_return)


👉 solving problem with n_horizon=50, n_states=2 n_inputs=2
🔄 it=0 	 (sub iter=140) 	 t_opt=0.5 	  eq=0.014530000000000001 	 neq=-0.0
🔄 it=1 	 (sub iter=134) 	 t_opt=0.8 	  eq=0.015960000000000002 	 neq=-0.0
🔄 it=2 	 (sub iter=157) 	 t_opt=1.28 	  eq=0.015850000000000003 	 neq=-0.0
🔄 it=3 	 (sub iter=120) 	 t_opt=2.05 	  eq=0.013810000000000001 	 neq=-0.0
🔄 it=4 	 (sub iter=203) 	 t_opt=3.2800000000000002 	  eq=0.01077 	 neq=-0.0
🔄 it=5 	 (sub iter=192) 	 t_opt=5.24 	  eq=0.007320000000000001 	 neq=-0.0
🔄 it=6 	 (sub iter=190) 	 t_opt=8.39 	  eq=0.005260000000000001 	 neq=-0.0
🔄 it=7 	 (sub iter=222) 	 t_opt=13.42 	  eq=0.00369 	 neq=-0.0
🔄 it=8 	 (sub iter=230) 	 t_opt=21.47 	  eq=0.00253 	 neq=-0.0
🔄 it=9 	 (sub iter=262) 	 t_opt=34.36 	  eq=0.00152 	 neq=-0.0
🔄 it=10 	 (sub iter=255) 	 t_opt=54.980000000000004 	  eq=0.00109 	 neq=-0.0
🔄 it=11 	 (sub iter=291) 	 t_opt=87.96000000000001 	  eq=0.00064 	 neq=-0.0
🔄 it=12 	 (sub iter=369) 	 t_opt=140.74 	  eq=0.00038 	 neq=-0.0
🔄 it=13 	 